# Cell cycle model CCCMV001

**Cell cycle model (variable mass) for mammalians ver. 001**

Packages

In [30]:
using OrdinaryDiffEq, ParameterizedFunctions, NamedTuples
using PyDSTool, PyCall, PyPlot
using JLD

In [25]:
include("custom_functions.jl");
include("plot_functions.jl");

In [26]:
@load "CCCMV001_data.jld"

2-element Array{Symbol,1}:
 :curve_lcycle
 :curve_stst  

### Model system of ODE's

Model function

In [27]:
f = @ode_def_bare CCMMV001 begin
    ## System of ODE (vector field)
    dMPF     = k_m0*Mass - (k_m2 + k_m2a*Wee1)*MPF + (k_m1 + k_m1a*Cdc25_P)*MPF_P - (k_m3 + k_m3a*APC_A)*MPF
    dMPF_P   = (k_m2 + k_m2a*Wee1)*MPF - (k_m1 + k_m1a*Cdc25_P)*MPF_P - (k_m3 + k_m3a*APC_A)*MPF_P
    dWee1    = V_w1*(Wee1_T - Wee1)/(J_w1 + Wee1_T - Wee1) - k_w2*(MPF + MPF_P*α)*Wee1/(J_w2 + Wee1)
    dCdc25_P = k_c1*(MPF + MPF_P*α)*(Cdc25_T - Cdc25_P)/(J_c1 + Cdc25_T - Cdc25_P) - V_c2*Cdc25_P/(J_c2 + Cdc25_P)
    dIE_A    = k_i1*(MPF + MPF_P*α)*(IE_T - IE_A)/(J_i1 + IE_T - IE_A) - V_i2*IE_A/(J_i2 + IE_A)
    dAPC_A   = k_a1*IE_A*(APC_T - APC_A)/(J_a1 + APC_T - APC_A) - V_a2*APC_A/(J_a2 + APC_A)
    dMass    = μ*Mass*(1 - Mass/K_Mass)
end k_m0=>0.2 k_m1=0.4 k_m1a=20.0 k_m2=0.4 k_m2a=50.0 k_m3=0.2 k_m3a=6.35 V_w1=1.0 k_w2=5.3 J_w1=0.01 J_w2=0.01 k_c1=8.5 V_c2=>1.4 k_c2a=2.2 J_c1=0.01 J_c2=0.01 k_i1=1.7 V_i2=0.4 J_i1=0.001 J_i2=0.001 k_a1=6.8 V_a2=1.7 J_a1=0.001 J_a2=0.001 α=0.05 μ=0.07 K_Mass=2.7 Wee1_T=1.0 Cdc25_T=1.0 IE_T=1.0 APC_T=1.0 Chk2_T=1.0 theta_M=>0.2

LoadError: [91mUndefVarError: @ode_def_bare not defined[39m

Cell division events

In [28]:
function division(t,u,integrator)
    ## Parameters
    ## Set of condition: MPF crosses threshold from positive to negative
    u[1] - f.theta_M
end # ModelCond

function division_event!(integrator)
    # Divide mass in two
    integrator.u[7] = integrator.u[7]/2
end # ModelEvent!

cb = ContinuousCallback(division, nothing, division_event!, save_positions=(true,false))

DiffEqBase.ContinuousCallback{#division,Void,#division_event!,DiffEqBase.#INITIALIZE_DEFAULT,Float64,Int64,Void}(division, nothing, division_event!, DiffEqBase.INITIALIZE_DEFAULT, nothing, true, 10, (true, false), 1.0e-9, 0)

### Important solutions

**Cell cycle**. Initial conditions and parameter options for normal cell cycle.

In [29]:
f.V_c2 = 1.4
u0 = [0.039,      # MPF
      1.660,      # MPF_P
      0.982,      # Wee1
      0.027,      # Cdc25_P
      0.001,      # IE_A
      0.000,      # APC_A
      1.938]      # Mass
tspan = (0.0, 28.0)
dtmax = 0.001
alg = Rosenbrock23()
prob = ODEProblem(f, u0, tspan)
sol = solve(prob, alg, callback=cb; (:dtmax, dtmax))
sol.errors

LoadError: [91mUndefVarError: f not defined[39m

In [7]:
sol

LoadError: [91mUndefVarError: sol not defined[39m

In [8]:
plot(sol.t, sol.u, lw=2)
legend(labels=["MPF","MPF_P"])

LoadError: [91mUndefVarError: plot not defined[39m

**Steady state**. Changing $V_{c2}$ to 4.0. This moves the dynamics to checkpoint, i.e., steady state solution.

In [9]:
f.V_c2 = 4.0
u0 = [0.042,      # MPF
      2.656,      # MPF_P
      0.886,      # Wee1
      0.005,      # Cdc25_P
      0.002,      # IE_A
      0.000,      # APC_A
      2.699]      # Mass
tspan=(0.0, 20.0)
prob = ODEProblem(f, u0, tspan)
dtmax = 0.1
sol = solve(prob, alg, callback=cb; (:dtmax, dtmax))
sol.errors

LoadError: [91mUndefVarError: f not defined[39m

In [10]:
plot(sol.t, sol.u, lw=2)
legend(labels=["MPF","MPF_P"])

LoadError: [91mUndefVarError: plot not defined[39m

Value of solution at the stady state for $V_{c2}$

In [11]:
sol.u[end]

LoadError: [91mUndefVarError: sol not defined[39m

### Find limit cycle

Functions

In [12]:
function find_cell_cycle(f, u0, tspan; alg=Rosenbrock23(), kwargs...)
    function division_stop!(integrator)
        integrator.u[7] = integrator.u[7]/2
        terminate!(integrator)
    end
    cb_start = ContinuousCallback(division, nothing, division_stop!)
    # Integrate until first division
    # NOTE: assert tspan is larger than needed
    prob = ODEProblem(f, u0, tspan)
    sol = solve(prob, alg, callback=cb_start; kwargs...)
    # One full cycle
    u0_start = sol.u[end]
    prob = ODEProblem(f, u0_start, tspan)
    sol = solve(prob, alg, callback=cb_start; kwargs...)
    # Done
    sol
end #function find_cell_cycle

find_cell_cycle (generic function with 1 method)

A full limit cycle starts and finishes on cell division (when MPF concentration drops bellow 0.2).

In [13]:
f.V_c2 = 1.6
u0 = [0.039,      # MPF
      1.660,      # MPF_P
      0.982,      # Wee1
      0.027,      # Cdc25_P
      0.001,      # IE_A
      0.000,      # APC_A
      1.938]      # Mass
tspan = (0.0, 28.0)
dtmax = 0.001
sol = find_cell_cycle(f, u0, tspan; (:dtmax, dtmax))
cycle = sol
sol.errors

LoadError: [91mUndefVarError: f not defined[39m

In [14]:
plot(sol.t, sol.u, lw=2)

LoadError: [91mUndefVarError: plot not defined[39m

Period

In [15]:
sol.t[end]-sol.t[1]

LoadError: [91mUndefVarError: sol not defined[39m

## Bifurcation

**Stable node and saddle point (saddle-node branch of the SNIC)**

In [16]:
u0 = [0.0425188,
      2.65499,
      0.887348,
      0.00584123,
      0.00290841,
      1.17587e-5,
      2.69903]
tspan = [0, 40]
dsargs = build_ode(f,u0,tspan)
dsargs[:pdomain] = Dict("V_c2"=>[0, 6])
ode = ds[:Generator][:Vode_ODEsystem](dsargs)
ode[:set](pars = Dict("V_c2"=>4.0))
PC = ds[:ContClass](ode)

LoadError: [91mUndefVarError: build_ode not defined[39m

In [17]:
name1 = "EQ1"
PCargs = ds[:args](name = name1)
PCargs[:type] = "EP-C"
PCargs[:freepars] = ["V_c2"]
PCargs[:MaxNumPoints] = 100
PCargs[:MaxStepSize] = 1e-0
PCargs[:MinStepSize] = 1e-5
PCargs[:StepSize] = 1e-2
PCargs[:LocBifPoints] = "all"
PCargs[:SaveEigen] = true
PCargs[:verbosity] = 2
PCargs[:StopAtPoints] = ["B"]
PC[:newCurve](PCargs)

LoadError: [91mUndefVarError: ds not defined[39m

In [18]:
# Commented to save computation, load "CCCMV001_data.jld"
#PC[:curves][name1][:backward]()
#PC[:curves][name1][:forward]()
#curve_stst = bifurcation_curve_ccc(PC, "EQ1");

**Limit cycle (oscilatory part of the SNIC)**

Now we follow the limit cycle

In [19]:
function follow_limit_cycle(initcycle, pdomain, stepsize; kwargs...)
    cycle = deepcopy(initcycle)
    f = cycle.prob.f
    # Start
    minmax = extrema(cycle[1, :])
    T = cycle.t[end] - cycle.t[1]
    out = [[f.V_c2, minmax[1], minmax[2], T]]
    # Forward
    while (f.V_c2 < (pdomain[2] - stepsize))
        f.V_c2 += stepsize
        u0 = cycle.u[end]
        tspan = (0.0, 1.3*T)
        cycle = find_cell_cycle(f, u0, tspan; kwargs...)
        minmax = extrema(cycle[1, :])
        T = cycle.t[end] - cycle.t[1]
        push!(out, [f.V_c2, minmax[1], minmax[2], T])
    end
    # Backward
    cycle = deepcopy(initcycle)
    f = cycle.prob.f
    while ((pdomain[1] + stepsize) < f.V_c2)
        f.V_c2 -= stepsize
        u0 = cycle.u[end]
        tspan = (0.0, 1.5*T)
        cycle = find_cell_cycle(f, u0, tspan; kwargs...)
        minmax = extrema(cycle[1, :])
        push!(out, [f.V_c2, minmax[1], minmax[2], cycle.t[end] - cycle.t[1]])
    end
    temp = sort(out, by = x -> x[1])
    temp = hcat(temp...)
    dict1 = Dict([:min=>temp[2,:], :max=>temp[3,:], :period=>temp[4,:]])
    dict2 = OrderedDict(Dict([:V_c2=>temp[1,:], :MPF=>dict1]))
    BifurcationCurve(dict2, repeat(["LC"], inner=[length(out)]), Dict{String,Any}(), Int[])
end

follow_limit_cycle (generic function with 1 method)

In [20]:
# Commented to save computation, load "CCCMV001_data.jld"
#dtmax = 0.001
#curve_lcycle = follow_limit_cycle(cycle, [0.001, 3.469], 0.01; (:dtmax, dtmax))

In [21]:
@save "CCCMV001_data.jld" curve_stst curve_lcycle

LoadError: [91mUndefVarError: @save not defined[39m

### Plot bifurcation diagram

In [22]:
PlotBifurcation(curve_stst, (:V_c2, :MPF))
PlotBifurcation(curve_lcycle, (:V_c2, :MPF))
xlim([0, 6])
xlabel(L"V_{c2}")
ylabel(L"MPF")
yscale("log")
ylim([0.001, 10])
title(L"Bifurcation diagram for $V_{c2}$")
show()

LoadError: [91mUndefVarError: PlotBifurcation not defined[39m

The stable steady state observed before corresponds to the stable branch of the the saddle-node bifurcation observed here. The bifurcation values is $V_{c2}^*=3.47168339841$.

In [23]:
curve_stst.special_points["LP"][1][:V_c2]

LoadError: [91mUndefVarError: curve_stst not defined[39m